In [ ]:
# !pip install datasets

# !pip install simpletransformers

# !pip install --upgrade accelerate

# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [ ]:
from typing import Dict, Any
from datasets import load_dataset, concatenate_datasets
# import evaluate

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PREPARING DATASET

In [ ]:
"""A large crowd-sourced dataset for developing natural language interfaces for relational databases"""


import json
import os

import datasets


In [ ]:
_DESCRIPTION = """\
A large crowd-sourced dataset for developing natural language interfaces for relational databases
"""

_AGG_OPS = ["", "MAX", "MIN", "COUNT", "SUM", "AVG"]
_COND_OPS = ["=", ">", "<", "OP"]

In [ ]:
def _convert_to_human_readable(sel, agg, columns, conditions):


        rep = f"SELECT {_AGG_OPS[agg]} {columns[sel] if columns is not None else f'col{sel}'} FROM table"

        if conditions:
            rep += " WHERE " + " AND ".join([f"{columns[i]} {_COND_OPS[o]} {v}" for i, o, v in conditions])
        return " ".join(rep.split())

def _generate_examples(main_filepath, tables_filepath):
  """Yields examples."""

  # Build dictionary to table_ids:tables
  with open(tables_filepath, encoding="utf-8") as f:
      tables = [json.loads(line) for line in f]
      id_to_tables = {x["id"]: x for x in tables}
  rows = []
  ids =[]
  with open(main_filepath, encoding="utf-8") as f:
      for idx, line in enumerate(f):
          row = json.loads(line)
          row["table"] = id_to_tables[row["table_id"]]
          del row["table_id"]

          # Handle missing data
          row["table"]["page_title"] = row["table"].get("page_title", "")
          row["table"]["section_title"] = row["table"].get("section_title", "")
          row["table"]["caption"] = row["table"].get("caption", "")
          row["table"]["name"] = row["table"].get("name", "")
          row["table"]["page_id"] = str(row["table"].get("page_id", ""))

          # Fix row types
          row["table"]["rows"] = [[str(e) for e in r] for r in row["table"]["rows"]]

          # Get human-readable version
          row["sql"]["human_readable"] = _convert_to_human_readable(
              row["sql"]["sel"],
              row["sql"]["agg"],
              row["table"]["header"],
              row["sql"]["conds"],
          )

          # Restructure sql->conds
          # - wikiSQL provides a tuple [column_index, operator_index, condition]
          #   as 'condition' can have 2 types (float or str) we convert to dict
          for i in range(len(row["sql"]["conds"])):
              row["sql"]["conds"][i] = {
                  "column_index": row["sql"]["conds"][i][0],
                  "operator_index": row["sql"]["conds"][i][1],
                  "condition": str(row["sql"]["conds"][i][2]),
              }
              rows.append(row)
              ids.append(i)
      return rows


In [ ]:
main_filepath = "/content/drive/MyDrive/wikisqldata/train.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/train.tables.jsonl"
train_da =_generate_examples(main_filepath, tables_filepath)
main_filepath = "/content/drive/MyDrive/wikisqldata/dev.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/dev.tables.jsonl"
val_da =_generate_examples(main_filepath, tables_filepath)
main_filepath = "/content/drive/MyDrive/wikisqldata/test.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/test.tables.jsonl"
test_da =_generate_examples(main_filepath, tables_filepath)

In [ ]:
print(len(train_da))
print(len(val_da))
print(len(test_da))

76729
11545
21846


In [ ]:
import pandas as pd
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=train_da))
validation_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=val_da))
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=test_da))

In [ ]:
train_dataset

Dataset({
    features: ['phase', 'question', 'sql', 'table'],
    num_rows: 76729
})

In [ ]:
from datasets.dataset_dict import DatasetDict

In [ ]:
dataset = DatasetDict({'train':train_dataset,'validation':validation_dataset,'test':test_dataset})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['phase', 'question', 'sql', 'table'],
        num_rows: 76729
    })
    validation: Dataset({
        features: ['phase', 'question', 'sql', 'table'],
        num_rows: 11545
    })
    test: Dataset({
        features: ['phase', 'question', 'sql', 'table'],
        num_rows: 21846
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['phase', 'question', 'sql', 'table'],
    num_rows: 76729
})

In [ ]:
# Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [ ]:
table_prefix = "table:"
question_prefix = "question:"

def preprocess_function(examples: Dict[str, Any]):
    """preprocess each row of wikisql datasets by create input with this format
        {question_prefix} {natural_question} {table_prefix} {table_schema}
        the labels will be the SQL statement

    Args:
        examples (Dict[str, Any]): each row of datasets

    Returns:
        output from tokenizer
    """
    columns_merge = [",".join(table["header"]) for table in examples["table"]]
    question_list = [question.replace(u'\xa0', u' ') for question in examples["question"]]
    assert len(columns_merge) == len(question_list)
    inputs = [f"{question_prefix} {question_list[i]} {table_prefix} {columns_merge[i]}" for i in range(len(columns_merge))]
    targets = [sql["human_readable"] for sql in examples["sql"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

In [ ]:
# run preprocess data
train_dataset = dataset["train"].map(preprocess_function, batched=True, remove_columns=["phase", "question", "table", "sql"])
test_dataset = dataset["test"].map(preprocess_function, batched=True, remove_columns=["phase", "question", "table", "sql"])
val_dataset = dataset["validation"].map(preprocess_function, batched=True, remove_columns=["phase", "question", "table", "sql"])

Map:   0%|          | 0/76729 [00:00<?, ? examples/s]

Map:   0%|          | 0/21846 [00:00<?, ? examples/s]

Map:   0%|          | 0/11545 [00:00<?, ? examples/s]

In [ ]:
# Training

In [ ]:
# declare model
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = model.cuda()

In [ ]:
table_prefix = "table:"
question_prefix = "question:"

def preprocess_function(examples: Dict[str, Any]):
    """preprocess each row of wikisql datasets by create input with this format
        {question_prefix} {natural_question} {table_prefix} {table_schema}
        the labels will be the SQL statement

    Args:
        examples (Dict[str, Any]): each row of datasets

    Returns:
        output from tokenizer
    """
    columns_merge = [",".join(table["header"]) for table in examples["table"]]
    question_list = [question.replace(u'\xa0', u' ') for question in examples["question"]]
    assert len(columns_merge) == len(question_list)
    inputs = [f"{question_prefix} {question_list[i]} {table_prefix} {columns_merge[i]}" for i in range(len(columns_merge))]
    targets = [sql["human_readable"] for sql in examples["sql"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

In [ ]:
print(type(dataset["train"]))
print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
# declare data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# declare training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    # gradient_checkpointing=True,
    warmup_ratio=0.01,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    fp16=True,
    predict_with_generate=True,
    # generation_max_length=512,
    # generation_num_beams=None,
    lr_scheduler_type="cosine",
    # dataloader_num_workers=2,
    greater_is_better=False,
    metric_for_best_model="eval_loss",
)

In [ ]:
# import numpy as np


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Decode generated summaries into text
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     # Decode reference summaries into text
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # ROUGE expects a newline after each sentence
#     decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
#     # Compute ROUGE scores
#     result = rouge_score.compute(
#         predictions=decoded_preds, references=decoded_labels, use_stemmer=True
#     )
#     # Extract the median scores
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# declare trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
) # you can evaluate by using compute_metrics function above, but I comment out for the faster training loop

In [ ]:
trainer.train()
# trainer.train(resume_from_checkpoint="./results/checkpoint-13000")

Step,Training Loss,Validation Loss
1000,0.000000,nan
2000,0.000000,nan
3000,0.000000,nan
4000,0.000000,nan


Step,Training Loss,Validation Loss
1000,0.000000,nan
2000,0.000000,nan
3000,0.000000,nan
4000,0.000000,nan


TrainOutput(global_step=4796, training_loss=0.0, metrics={'train_runtime': 3293.0506, 'train_samples_per_second': 46.601, 'train_steps_per_second': 1.456, 'total_flos': 1.4584895609296896e+16, 'train_loss': 0.0, 'epoch': 2.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Sandeep_text_SQL_customdataset")

In [ ]:
# Testing

In [ ]:
from typing import List

table_prefix = "table:"
question_prefix = "question:"

def prepare_input(question: str, table: List[str]):
    print("question:", question)
    print("table:", table)
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=512)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

In [ ]:
test_id = 1000
print("model result:", inference(dataset["test"][test_id]["question"], dataset["test"][test_id]["table"]["header"]))
print("real result:", dataset["test"][test_id]["sql"]["human_readable"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


question: what's the regionalliga nord with regionalliga west/südwest being fc gütersloh rot-weiß essen
table: ['Season', 'Regionalliga Süd', 'Regionalliga West/Südwest', 'Regionalliga Nord', 'Regionalliga Nord-Ost']
model result: season, Regionalliga Süd, Regionalliga West/Südwest, Regionalliga Nord, Regionalliga Nord-Ost
real result: SELECT Regionalliga Nord FROM table WHERE Regionalliga West/Südwest = FC Gütersloh Rot-Weiß Essen


In [ ]:
inference("what is id with name jui and age equal 25", ["id","name", "age"])

question: what is id with name jui and age equal 25
table: ['id', 'name', 'age']


'name jui and age equal 25 table: id name jui with name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui, age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age equal 25 table: id name jui and age

In [ ]:
inference("get people name with age equal 25", ["id","name", "age"])

question: get people name with age equal 25
table: ['id', 'name', 'age']


'get people name with age equal 25 table: id,name,age'

In [ ]:
# Testing Model

In [ ]:
# declare model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Sandeep_text_SQL_customdataset")
model = model.cuda()
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Sandeep_text_SQL_customdataset")

In [ ]:
from typing import List

table_prefix = "table:"
question_prefix = "question:"

def prepare_input(question: str, table: List[str]):
    print("question:", question)
    print("table:", table)
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=512)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

In [ ]:
test_id = 1000
print("model result:", inference(dataset["test"][test_id]["question"], dataset["test"][test_id]["table"]["header"]))
print("real result:", dataset["test"][test_id]["sql"]["human_readable"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


question: what's the regionalliga nord with regionalliga west/südwest being fc gütersloh rot-weiß essen
table: ['Season', 'Regionalliga Süd', 'Regionalliga West/Südwest', 'Regionalliga Nord', 'Regionalliga Nord-Ost']
model result: regionalliga nord with regionalliga west/südwest being fc gütersloh rot-weiß essen table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regionalliga Süd,Regionalliga West/Südwest,Regionalliga Nord-Ost table: Season,Regi

In [ ]:
inference("what is id with name jui and age equal 25", ["id","name", "age"])

question: what is id with name jui and age equal 25
table: ['id', 'name', 'age']


'id with name jui and age equal 25 - id with name jui and age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with age equal 25 - id with